<a href="https://colab.research.google.com/github/alexmaks/n8n/blob/master/InstantID_IPAdapter_ControlNet_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b totoro https://github.com/camenduru/ComfyUI /content/TotoroUI
!git clone -b totoro_v2 https://github.com/camenduru/ComfyUI_IPAdapter_plus /content/TotoroUI/IPAdapter
!git clone -b totoro https://github.com/camenduru/ComfyUI_InstantID /content/TotoroUI/InstantID

!pip install -q torch==2.2.1+cu121 torchvision==0.17.1+cu121 torchaudio==2.2.1+cu121 torchtext==0.17.1 torchdata==0.7.1 --extra-index-url https://download.pytorch.org/whl/cu121
!pip install -q torchsde einops diffusers accelerate xformers==0.0.25 insightface onnxruntime onnxruntime-gpu
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/470847 -d /content/TotoroUI/models -o raemuXL_v35Lightning.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -d /content/TotoroUI/models/clip_vision -o CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors  -d /content/TotoroUI/models/ipadapter -o ip-adapter-plus-face_sdxl_vit-h.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose.safetensors -d /content/TotoroUI/models/controlnet -o thibaud_xl_openpose.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/Ttl/ComfyUi_NNLatentUpscale/raw/master/sdxl_resizer.pt -d /content/TotoroUI/models -o sdxl_resizer.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/DIAMONIK7777/antelopev2/resolve/main/1k3d68.onnx -d /content/TotoroUI/models/insightface/models/antelopev2 -o 1k3d68.onnx
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/DIAMONIK7777/antelopev2/resolve/main/2d106det.onnx -d /content/TotoroUI/models/insightface/models/antelopev2 -o 2d106det.onnx
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/DIAMONIK7777/antelopev2/resolve/main/genderage.onnx -d /content/TotoroUI/models/insightface/models/antelopev2 -o genderage.onnx
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/DIAMONIK7777/antelopev2/resolve/main/glintr100.onnx -d /content/TotoroUI/models/insightface/models/antelopev2 -o glintr100.onnx
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/DIAMONIK7777/antelopev2/resolve/main/scrfd_10g_bnkps.onnx -d /content/TotoroUI/models/insightface/models/antelopev2 -o scrfd_10g_bnkps.onnx
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/InstantX/InstantID/resolve/main/ip-adapter.bin -d /content/TotoroUI/models/instantid -o ip-adapter.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/InstantX/InstantID/resolve/main/ControlNetModel/diffusion_pytorch_model.safetensors -d /content/TotoroUI/models/controlnet/SDXL/instantid -o diffusion_pytorch_model.safetensors

!wget https://huggingface.co/camenduru/IICF/resolve/main/test/anya.jpg -O /content/anya.jpg
!wget https://huggingface.co/camenduru/IICF/resolve/main/test/pose_images.zip -O /content/pose_images.zip
!unzip /content/pose_images.zip

/content
Cloning into '/content/TotoroUI'...
remote: Enumerating objects: 20484, done.
remote: Total 20484 (delta 0), reused 0 (delta 0), pack-reused 20484 (from 1)
Receiving objects: 100% (20484/20484), 71.55 MiB | 15.50 MiB/s, done.
Resolving deltas: 100% (13651/13651), done.
Cloning into '/content/TotoroUI/IPAdapter'...
remote: Enumerating objects: 733, done.
remote: Counting objects: 100% (376/376), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 733 (delta 356), reused 270 (delta 270), pack-reused 357 (from 1)
Receiving objects: 100% (733/733), 4.71 MiB | 12.01 MiB/s, done.
Resolving deltas: 100% (490/490), done.
Cloning into '/content/TotoroUI/InstantID'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 160 (delta 68), reused 64 (delta 64), pack-reused 87 (from 1)
Receiving objects: 100% (160/160), 1.61 MiB | 4.51 MiB/s, done.
Resolving deltas: 100% (95/95), 

In [9]:
%cd /content/TotoroUI
import torch
import numpy as np
from PIL import Image
import totoro
import nodes
import sys
sys.path.append('/content/TotoroUI/IPAdapter')
import IPAdapterPlus
sys.path.append('/content/TotoroUI/InstantID')
import InstantID
import scipy
import model_management
from latent_resizer import LatentResizer
from totoro import model_management
import gc
import random

def upscale(latent, upscale):
  device = model_management.get_torch_device()
  samples = latent.to(device=device, dtype=torch.float16)
  model = LatentResizer.load_model('/content/TotoroUI/models/sdxl_resizer.pt', device, torch.float16)
  model.to(device=device)
  latent_out = (model(0.13025 * samples, scale=upscale) / 0.13025)
  latent_out = latent_out.to(device="cpu")
  model.to(device=model_management.vae_offload_device())
  return ({"samples": latent_out},)

with torch.no_grad():
    model_patcher, clip, vae, clipvision = totoro.sd.load_checkpoint_guess_config("/content/TotoroUI/models/raemuXL_v35Lightning.safetensors", output_vae=True, output_clip=True, embedding_directory=None)
    IPAdapterPlus_model = IPAdapterPlus.IPAdapterUnifiedLoader().load_models(model_patcher, 'PLUS FACE (portraits)', lora_strength=0.0, provider="CUDA")
    instantid = InstantID.InstantIDModelLoader().load_model("/content/TotoroUI/models/instantid/ip-adapter.bin")
    insightface = InstantID.InstantIDFaceAnalysis().load_insight_face("CUDA")
    instantid_control_net = totoro.controlnet.load_controlnet("/content/TotoroUI/models/controlnet/SDXL/instantid/diffusion_pytorch_model.safetensors")
    output_image, output_mask = nodes.LoadImage().load_image("/content/anya.jpg")
    image_kps, image_kps_mask = nodes.LoadImage().load_image("/content/pose_images/pose_kps_00008_.png")
    image_dw, image_dw_mask = nodes.LoadImage().load_image("/content/pose_images/pose_dw_pose_00008_.png")
    ip_model_patcher = IPAdapterPlus.IPAdapterAdvanced().apply_ipadapter(IPAdapterPlus_model[0], IPAdapterPlus_model[1], image=output_image, weight_type="style transfer")
    tokens = clip.tokenize("1girl")
    cond, pooled = clip.encode_from_tokens(tokens, return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    n_tokens = clip.tokenize("(nsfw:1.5), nipple, nude, naked, lowres, child, getty, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, artist name, trademark, watermark, title, multiple view, reference sheet, mutated hands and fingers, poorly drawn face, mutation, deformed, ugly, bad proportions, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, tatoo, amateur drawing, odd eyes, uneven eyes, unnatural face, uneven nostrils, crooked mouth, bad teeth, crooked teeth, photoshop, video game, censor, censored, ghost, b&w, weird colors, gradient background, spotty background, blurry background, ugly background, simple background, realistic, out of frame, extra objects, gross, ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of focus, blurry, very long body, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn eyes, cloned face, disfigured, deformed, cross-eye, extra limbs, missing limb, malformed hands, mutated, morbid, mutilated, disfigured, extra arms, extra hands, mangled fingers, contorted, conjoined, mismatched limbs, mismatched parts, bad perspective, black and white, oversaturated, undersaturated, bad shadow, cropped image, draft, grainy, pixelated")
    n_cond, n_pooled = clip.encode_from_tokens(n_tokens, return_pooled=True)
    n_cond = [[n_cond, {"pooled_output": n_pooled}]]
    work_model, instantid_cond, instantid_n_cond = InstantID.ApplyInstantID().apply_instantid(instantid=instantid[0], insightface=insightface[0], control_net=instantid_control_net, image=output_image, model=ip_model_patcher[0], positive=cond, negative=n_cond, start_at=0.0, end_at=1.0, weight=0.80, image_kps=image_kps)
    openpose_control_net = totoro.controlnet.load_controlnet("/content/TotoroUI/models/controlnet/thibaud_xl_openpose.safetensors")
    openpose_cond = nodes.ControlNetApply().apply_controlnet(conditioning=instantid_cond, control_net=openpose_control_net, image=image_dw, strength=0.90)

/content/TotoroUI


ModuleNotFoundError: No module named 'torchsde'

In [10]:
instantid = None
insightface = None
instantid_control_net = None
ip_model_patcher = None
cond = None
n_cond = None
model_patcher = None
clip = None
clipvision = None
IPAdapterPlus_model = None
model_management.cleanup_models()
gc.collect()
model_management.soft_empty_cache()

ran = random.randint(0, 65535)
print(ran)

with torch.no_grad():
    latent = {"samples":torch.zeros([1, 4, 1024 // 8, 1024 // 8])}
    sample = nodes.common_ksampler(model=work_model,
                          seed=ran,
                          steps=4,
                          cfg=1.3,
                          sampler_name="dpmpp_sde_gpu",
                          scheduler="karras",
                          positive=openpose_cond[0],
                          negative=instantid_n_cond,
                          latent=latent,
                          denoise=0.95)

    with torch.inference_mode():
        sample = sample[0]["samples"].to(torch.float16)
        vae.first_stage_model.cuda()
        decoded = vae.decode_tiled(sample).detach()

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])

NameError: name 'model_management' is not defined

In [ ]:
with torch.no_grad():
  latent = upscale(sample, 1.5)
  sample = nodes.common_ksampler(model=work_model,
                            seed=ran,
                            steps=4,
                            cfg=1.3,
                            sampler_name="dpmpp_sde_gpu",
                            scheduler="karras",
                            positive=openpose_cond[0],
                            negative=instantid_n_cond,
                            latent=latent[0],
                            denoise=0.55)
  with torch.inference_mode():
    sample = sample[0]["samples"].to(torch.float16)
    vae.first_stage_model.cuda()
    decoded = vae.decode_tiled(sample).detach()
Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])